In [ ]:
!pip install timm

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
import pandas as pd
from timm import create_model
import os


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [4]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [5]:
cifar10_test = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

fine_tune_ratio = 0.2
cifar10_fine_tune_size = int(fine_tune_ratio * len(cifar10_test))
cifar10_eval_size = len(cifar10_test) - cifar10_fine_tune_size
cifar10_fine_tune, cifar10_eval = random_split(cifar10_test, [cifar10_fine_tune_size, cifar10_eval_size])

100%|██████████| 170M/170M [00:03<00:00, 43.9MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data


In [6]:
batch_size = 32
cifar10_fine_tune_loader = DataLoader(cifar10_fine_tune, batch_size=batch_size, shuffle=True)
cifar10_eval_loader = DataLoader(cifar10_eval, batch_size=batch_size, shuffle=False)

In [7]:
models = {
    "ViT-B-16": create_model('vit_base_patch16_224', pretrained=True),
    "ViT-L-16": create_model('vit_large_patch16_224', pretrained=True),
    "DeiT-B": create_model('deit_base_patch16_224', pretrained=True),
}

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

In [9]:
num_classes_cifar10 = 10
for model_name, model in models.items():
    model.head = torch.nn.Linear(model.head.in_features, num_classes_cifar10)
    model.to(device)

In [ ]:
def fine_tune_model(model, data_loader, num_epochs=2, learning_rate=1e-4):
    
    for param in model.parameters():
        param.requires_grad = False
    for param in model.head.parameters():
        param.requires_grad = True

    
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.head.parameters(), lr=learning_rate)

   
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(data_loader):.4f}")


In [11]:
for model_name, model in models.items():
    print(f"Fine-tuning {model_name} on CIFAR-10...")
    fine_tune_model(model, cifar10_fine_tune_loader)

Fine-tuning ViT-B-16 on CIFAR-10...
Epoch 1/2, Loss: 1.7836
Epoch 2/2, Loss: 1.0417
Fine-tuning ViT-L-16 on CIFAR-10...
Epoch 1/2, Loss: 1.8731
Epoch 2/2, Loss: 0.5834
Fine-tuning DeiT-B on CIFAR-10...
Epoch 1/2, Loss: 2.1283
Epoch 2/2, Loss: 1.6481


In [ ]:
def evaluate_and_save_results(model, data_loader, dataset_name, model_name, output_dir="./results"):
    """
    Evaluate the model and save results to a CSV file.
    """
    results = []
    correct = 0
    total = 0
    model.eval()
    with torch.no_grad():
        for batch_idx, (images, labels) in enumerate(data_loader):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            confidences = torch.softmax(outputs, dim=1).max(dim=1).values
            _, predicted = torch.max(outputs, 1)

            
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            
            for i in range(len(labels)):
                results.append({
                    'image_index': batch_idx * len(labels) + i,
                    'true_label': labels[i].item(),
                    'predicted_label': predicted[i].item(),
                    'confidence': confidences[i].item(),
                    'correct': labels[i].item() == predicted[i].item()
                })

    
    accuracy = 100 * correct / total
    print(f"Accuracy of {model_name} on {dataset_name}: {accuracy:.2f}%")

    
    df = pd.DataFrame(results)

    
    os.makedirs(output_dir, exist_ok=True)

    
    output_file = f"{output_dir}/{model_name}_{dataset_name}_results.csv"
    df.to_csv(output_file, index=False)
    print(f"Results saved to {output_file}")


In [13]:
dataset_name = "CIFAR-10"
for model_name, model in models.items():
    print(f"Evaluating {model_name} on {dataset_name}...")
    evaluate_and_save_results(model, cifar10_eval_loader, dataset_name, model_name)

Evaluating ViT-B-16 on CIFAR-10...
Accuracy of ViT-B-16 on CIFAR-10: 82.53%
Results saved to ./results/ViT-B-16_CIFAR-10_results.csv
Evaluating ViT-L-16 on CIFAR-10...
Accuracy of ViT-L-16 on CIFAR-10: 90.50%
Results saved to ./results/ViT-L-16_CIFAR-10_results.csv
Evaluating DeiT-B on CIFAR-10...
Accuracy of DeiT-B on CIFAR-10: 73.71%
Results saved to ./results/DeiT-B_CIFAR-10_results.csv
